## ETL
### Export, Transform & Load

Comienzo importando las librerias necesarias

In [1]:
import pandas as pd
import json

### Creamos el dataset de Games

Procedo a hacer el ETL (carga, transformacion y guardado) en un CSV.

In [24]:
### Se carga el JSON en un Dataframe ###

data = []
with open('output_steam_games.json') as e:
    for line in e:
        data.append(json.loads(line))
dfgames = pd.DataFrame(data)

Eliminacion de valores faltantes

In [25]:
dfgames = dfgames.dropna()

Repaso columnas analisandolas para saber cuale son indeseadas para su posterior eliminacion

In [26]:
dfgames.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

Procedo a eliminar columnas indeseadas

In [27]:
dfgames = dfgames.drop(columns=['developer', 'publisher', 'early_access', 'reviews_url', 'price', 'url', 'specs', 'tags', 'app_name'])

Creo dummies para la columna genero

In [28]:
### Primero que nada, rellenamos los valores faltantes con una lista vacia ###
dfgames['genres'] = dfgames['genres'].fillna('[]')

### Posteriormente convertimos la lista de generos a una lista separada por comas ###
dfgames['genres'] = dfgames['genres'].apply(lambda x: ', '.join(x))

### Creo los dummies para la columna 'genres' ###
dummy_genres = dfgames['genres'].str.get_dummies(', ')

### Lo fusiono con el Dataframe original ###
dfgames_dummies = pd.concat([dfgames, dummy_genres], axis=1)

In [29]:
dfgames = dfgames_dummies

Se modificó la columna 'release_date' a tipo year para trabajar facilmente con los años

In [30]:
### Se reemplazan los valores no validos por NaT ###
dfgames["release_date"] = pd.to_datetime(dfgames["release_date"], errors='coerce')

### Filtro el Dataframe solo por los valores validos ###
df_filtered = dfgames.dropna(subset=["release_date"])

### Convierto la columna 'release_date' a tipo Datetime ###
df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])

### Relleno los valores faltantes en las columna 'release_date' con la moda ###
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    dfgames[col].fillna(dfgames[col].mode()[0], inplace=True)

### Modifico y posteriormente elimino la columna 'release_date' ###
dfgames['year'] = dfgames['release_date'].dt.year.astype('Int64')

dfgames.drop('release_date', axis=1, inplace=True)

C:\Users\tiago\AppData\Local\Temp\ipykernel_5396\3726339654.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])


In [2]:
### Vemos como va quedando ordenado y limpio el Dataframe 'Games' ###
dfgames

NameError: name 'dfgames' is not defined

Las columnas finales

In [32]:
dfgames.columns

Index(['genres', 'title', 'id', 'Action', 'Adventure',
       'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing', 'year'],
      dtype='object')

Guardo el Dataframe en un CSV para un uso mas sencillo

In [ ]:
dfgames.to_csv('games_clean.csv')

### Datasets de Reviews

Se procede a hacer el ETL (carga, transformacion y guardado) en un CSV.

In [ ]:
with open("australian_user_reviews.json", 'r', encoding='utf-8') as w:
    data = w.readlines()
    
datos = [eval(line.strip()) for line in data]

df_reviews = pd.DataFrame(datos)
df_reviews.head()

### Desanidado de la columna Reviews

Primero creo un dataframe con el archivo review para acceder a la columna que me interesa

In [ ]:
df_reviews_open = df_reviews.explode('reviews')

### Con este bloque de codigo creo un nuevo dataframe con la info ya desanidada ###

dfreviewsopen = pd.concat([df_reviews_open.drop(['reviews'], axis=1), df_reviews_open['reviews'].apply(pd.Series)], axis=1)

Creo la columna 'posted year' para trabajar simplemente con el año

In [ ]:
dfreviewsopen['posted year'] = dfreviewsopen['posted'].str.extract(r'(\d{4})')

De paso elimino la columna 'posted' ya que no me sirve

In [ ]:
dfreviewsopen.drop('posted' , axis = 1, inplace = True)

In [ ]:
### Breve vistazo de como se ve el dataframe ###
dfreviewsopen.head(3)

### Implementando el Sentimental Score

In [ ]:

### Importacion de las librerias ###
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

### Inicializacion del analizador de sentimientos ### 
sia = SentimentIntensityAnalyzer()

### Definición de la función get_sentiment_score() ###
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  ### Retorna neutral si el texto esta vacio o es nulo ###
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 0  ### Buen puntaje ###
        elif compound_score <= -0.05:
            return 2  ### Mal puntaje ###
        else:
            return 1 ### Neutral ###
    else:
        return 1  ### Si el texto no es una cadena tambien devuelve 1 (neutral) ###


### Convierto la columna 'review' a tipo de dato 'str' ###
dfreviewsopen['review'] = dfreviewsopen['review'].astype(str)

### Aplico la función get_sentiment_score() a la columna 'review' ###
dfreviewsopen['sentiment_score'] = dfreviewsopen['review'].apply(get_sentiment_score)

### Imprimo el dataframe actualizado y finalizado ###
print(dfreviewsopen)

In [12]:
dfreviewsopen.to_csv('clean_reviews.csv')

### Dataset de Items

Se procede a hacer el ETL (carga, transformacion y guardado) en un CSV.

In [14]:
with open("australian_users_items.json", 'r', encoding='utf-8') as i:
    data = i.readlines()
    
records = [eval(line.strip()) for line in data]

item_df = pd.DataFrame(records)
item_df.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Hacemos un paneo para ver que items necesito

In [15]:
print(item_df['items'][0])

[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '60', 'item_name': 'Ricochet', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '70', 'item_name': 'Half-Life', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 4733, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_forever': 1853, 'playtime_2weeks': 0}, {'item_i

Desanidado de items

In [17]:
### Aplico la funcion lambda para desanidar las keys que busco con sus values ###
item_df['playtime_forever'] = item_df['items'].apply(lambda x: x[0].get('playtime_forever') if len(x) > 0 else 0)
item_df['playtime_2weeks'] = item_df['items'].apply(lambda x: x[0].get('playtime_2weeks') if len(x) > 0 else 0)
item_df['id'] = item_df['items'].apply(lambda x: x[0].get('item_id') if len(x) > 0 else 0)
item_df['playtime_forever'] = item_df['playtime_forever'].astype(int)
item_df['playtime_2weeks'] = item_df['playtime_2weeks'].astype(int)

Dropeo columnas que no voy a necesitar

In [18]:
item_df.drop(['user_url'], axis=1, inplace=True)
item_df.drop(['items'], axis=1, inplace=True)

In [19]:
item_df ### Veamos como va la tabla

,user_id,items_count,steam_id,playtime_forever,playtime_2weeks,id
0,76561197970982479,277,76561197970982479,6,0,10
1,js41637,888,76561198035864385,0,0,10
2,evcentric,137,76561198007712555,923,0,1200
3,Riot-Punch,328,76561197963445855,0,0,10
4,doctr,541,76561198002099482,1131,0,300
...,...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,0,0,413850
88306,76561198326700687,177,76561198326700687,0,0,11020
88307,XxLaughingJackClown77xX,0,76561198328759259,0,0,0
88308,76561198329548331,7,76561198329548331,677,677,304930


In [20]:
item_df.to_csv('clean_items.csv', index=False)

### Elaboracion del Dataset final

A traves de los ids podemos unir las tablas para ocupar menos espacio

In [34]:
merged_dataframe = pd.merge(dfgames, item_df, on='id') ### Se combinan las tablas ya desanidadas

In [35]:
merged_dataframe.drop(['genres'],axis=1, inplace=True) ### 'genres' ya no es necesario usarlo

In [36]:
merged_dataframe.shape ### Revisamos con que dimensiones queda finalmente

(62229, 29)

In [37]:
merged_dataframe.to_csv('Df_Final.csv', index=False)

In [38]:
df_reviews = pd.read_csv('clean_reviews.csv') ### Cargamos reviews para unirlo posteriormente

In [39]:
final_df = pd.merge(merged_dataframe, df_reviews, on = 'user_id') ### Se une el dataset combinado con el dataset Items

In [ ]:
final_df.to_csv('df_final.csv', index=False) ### Se guarda el dataset final

Revisamos columnas finales para verificar si hay algo por borrar

In [40]:
final_df.columns

Index(['title', 'id', 'Action', 'Adventure', 'Animation &amp; Modeling',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie',
       'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation',
       'Software Training', 'Sports', 'Strategy', 'Utilities',
       'Video Production', 'Web Publishing', 'year', 'user_id', 'items_count',
       'steam_id', 'playtime_forever', 'playtime_2weeks', 'Unnamed: 0',
       'user_url', 'funny', 'posted', 'last_edited', 'item_id', 'helpful',
       'recommend', 'review', '0', 'posted year', 'sentiment_score'],
      dtype='object')

Borramos columnas que se agregaron con el merge

In [41]:
final_df.drop(columns=['0', 'Unnamed: 0'], inplace=True)

### Dataset Finalizado

In [43]:
final_df.to_csv('df_final.csv', index=False)